# Imports

In [1]:
import datasets
from bs4 import BeautifulSoup
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import re
from evaluation import evaluate
import nltk
nltk.download('stopwords')

/home/ojasva20318/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-30 05:19:06.832617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 05:19:07.965254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ojasva20318/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Constants

In [2]:
PATH_TO_OWL = './LMSS.owl'
LLM_PATH = '../open-llama-7b-open-instruct'

# Set up Llama

In [3]:
tokenizer = AutoTokenizer.from_pretrained(LLM_PATH, use_fast=False)

model = AutoModelForCausalLM.from_pretrained(LLM_PATH, torch_dtype=torch.float16, device_map='sequential')

Loading checkpoint shards: 100%|██████████| 3/3 [01:03<00:00, 21.31s/it]


In [69]:
def get_llama_response(input_text, tokenizer, model):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    output1 = model.generate(input_ids, max_length=4096)
    input_length = input_ids.shape[1]
    output1 = output1[:, input_length:]
    output_text = tokenizer.decode(output1[0])
    return output_text

In [5]:
#example
prompt_template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:"

prompt = 'Is the following statement hearsay.\nStatement: I heard him talking to her using bad language.\nResponse should start with the word Yes/No followed by reasoning for the answer'


inputt = prompt_template.format(instruction= prompt)

print(inputt)
answer = get_llama_response(inputt,tokenizer,model)
print(answer)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Is the following statement hearsay.
Statement: I heard him talking to her using bad language.
Response should start with the word Yes/No followed by reasoning for the answer

### Response:
 Yes. It is hearsay because it is not being directly stated by the person who witnessed the event.</s>


# Reading OWL

In [6]:
# Read the contents of the .owl file
with open(PATH_TO_OWL, "r") as owl_file:
    owl_data = owl_file.read()

# Parse the OWL data using BeautifulSoup
soup = BeautifulSoup(owl_data, 'xml')

In [7]:
# Initialize lists to store data
labels = []
definitions = []

# Find all instances of <owl:Class> elements and extract label and definition
for owl_class in soup.find_all('owl:Class'):
    label_element = owl_class.find('rdfs:label')
    definition_element = owl_class.find('skos:definition')
    
    # Check if label and definition elements exist
    if label_element and definition_element:
        label = label_element.text.strip()
        definition = definition_element.text.strip()
        
        # Append data to lists
        labels.append(label)
        definitions.append(definition)

data = {'Label': labels, 'Definition': definitions}
owl_df = pd.DataFrame(data)

owl_df

,Label,Definition
0,Other Personal and Household Goods Repair and ...,See industry description for 811490.
1,Other Converted Paper Product Manufacturing,This industry comprises establishments primari...
2,General Medical and Surgical Hospitals,NULL
3,Confectionery Merchant Wholesalers,This industry comprises establishments primari...
4,Other Specialized Design Services,See industry description for 541490.
...,...,...
14248,Vocational Rehabilitation Services,NULL
14249,Books Printing,This U.S. industry comprises establishments pr...
14250,Petrochemical Manufacturing,See industry description for 325110.
14251,Pesticide and Other Agricultural Chemical Manu...,This industry comprises establishments primari...


### Function to get classes

In [8]:
import random


def filter_label_by_substring(df, substring):
    """
    Filter DataFrame rows containing the specified substring in the 'Label' column
    and return a list of strings in the format "{Label} : {Definition}".
    
    Args:
        df (pandas.DataFrame): Input DataFrame.
        substring (str): Substring to search for.
        
    Returns:
        list: List of strings in the format "{Label} : {Definition}" for matching rows.
    """
    filtered_df = df[df['Label'].str.contains(substring, case=False)]
    output_list = []
    
    if len(filtered_df) <= 3:
        for index, row in filtered_df.iterrows():
            output_list.append(f"{row['Label']} : {row['Definition']}")
    else:
        selected_indices = random.sample(range(len(filtered_df)), 3)
        for idx in selected_indices:
            row = filtered_df.iloc[idx]
            output_list.append(f"{row['Label']} : {row['Definition']}")
    
    return output_list

In [9]:
def filter_words_by_substring(words, df):
    """
    Filter DataFrame rows for each word in the list of words and append the results in a final list.
    
    Args:
        words (list): List of words.
        df (pandas.DataFrame): Input DataFrame.
        
    Returns:
        list: List of strings in the format "{Label} : {Definition}" for matching rows for all words.
    """
    final_output = []
    for word in words:
        output_list = filter_label_by_substring(df, word)
        final_output.extend(output_list)
    return final_output

In [12]:
from nltk.corpus import stopwords

def remove_stopwords(text, language='english'):
    # Get the stopwords for the specified language
    stopwords_list = set(stopwords.words(language))
    
    # Split the text into words
    words = text.split()
    
    # Remove stopwords
    filtered_words = [word for word in words if word.lower() not in stopwords_list]
    
    return filtered_words

In [15]:
#example usage
search_substring = 'On the issue of whether Jennifer suffered reputational harm from Lindas article, the fact that Linda worked with several different editors to proof read and cross-check her article.'
filtered_words = remove_stopwords(search_substring)
result = filter_words_by_substring(filtered_words,owl_df)
result.append(filter_label_by_substring(owl_df,'hearsay'))
print(result)
print(len(result))

['Analysis of Pre-Petition Lien Issues : Analysis of Pre-Petition Lien Issues refers to the examination and evaluation of any legal claims or encumbrances on property that arise before the filing of a bankruptcy petition.', 'Issuer : An issuer is a legal entity that develops, registers and sells securities to finance its operations. Issuers may be corporations, investment trusts, or domestic or foreign governments.', "Travel Document Issued Date : The Travel Document Issued Date is the specific date on which a travel document like a passport or visa was officially issued by the relevant authority. This date is important for determining the document's validity period and is often required when applying for visas or entering foreign countries", "Reputational Risk : The threats and opportunities associated with an organization's reputation and credibility with its partners, stakeholders, and the public — that might damage an organization's public image, credibility, or brand value. These 

# Dataset

In [16]:
dataset_hearsay = datasets.load_dataset("nguha/legalbench", "hearsay")

/home/ojasva20318/.local/lib/python3.8/site-packages/datasets/load.py:1454: FutureWarning: The repository for nguha/legalbench contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nguha/legalbench
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [17]:
test_df = dataset_hearsay['test'].to_pandas()
test_df

,answer,index,slice,text
0,No,0,Non-assertive conduct,On the issue of whether James is an smart indi...
1,No,1,Non-assertive conduct,On the issue of whether Robert negligently dro...
2,No,2,Non-assertive conduct,On the issue of whether John knew about the co...
3,No,3,Non-assertive conduct,On the issue of whether Michael was guilty of ...
4,No,4,Non-assertive conduct,On the issue of whether William was loved by h...
...,...,...,...,...
89,No,89,Not introduced to prove truth,To prove that Arthur believed that Amy and Dan...
90,No,90,Not introduced to prove truth,To prove that the trademarks of restaurant A a...
91,No,91,Not introduced to prove truth,To prove that Michael knew of the existing pat...
92,No,92,Not introduced to prove truth,To prove that Arthur and Mary had a conversati...


In [18]:
prompts = test_df["text"].tolist()
prompts

['On the issue of whether James is an smart individual, the fact that James came first in his class in law school.',
 'On the issue of whether Robert negligently drove, the fact that Robert fell asleep while driving.',
 'On the issue of whether John knew about the conspiracy, the fact that John likes sweatpants.',
 'On the issue of whether Michael was guilty of murder, the fact that Michael left the crime scene immediately.',
 'On the issue of whether William was loved by his community, the fact that he was selected to speak at his graduation.',
 'On the issue of whether Mary robbed the bank, the fact that Mary went to the bank in disguise.',
 'On the issue of whether Patricia was a fan of Coldplay, the fact that she had a poster with the lyrics of "Viva la Vida" on her bedroom wall.',
 "On the issue of whether Jennifer suffered reputational harm from Linda's article, the fact that Linda worked with several different editors to proof read and cross-check her article.",
 "On the issue o

In [19]:
prompts[0]

'On the issue of whether James is an smart individual, the fact that James came first in his class in law school.'

In [15]:
def read_tsv(file_path):
    examples = []
    with open(file_path, 'r') as file:
        next(file)
        for line in file:
            index, answer, text, _ = line.strip().split('\t')
            examples.append((text, answer))
    return examples

# Read examples from test.tsv
examples = read_tsv('tasks/hearsay/train.tsv')

print(examples)

[('On the issue of whether David is fast, the fact that David set a high school track record.', 'No'), ('On the issue of whether Rebecca was ill, the fact that Rebecca told Ronald that she was unwell.', 'Yes'), ('"To prove that Tim was a soccer fan, the fact that Tim told Jimmy that ""Real Madrid was the best soccer team in the world."""', 'No'), ('"When asked by the attorney on cross-examination, Alice testified that she had ""never seen the plaintiff before, and had no idea who she was."""', 'No'), ('On the issue of whether Martin punched James, the fact that Martin smiled and nodded when asked if he did so by an officer on the scene.', 'Yes')]


In [59]:
def add_labels_and_definitions_to_prompt(prompt_text):
    # Initialize full_prompt with prompt_text
    full_prompt = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.\n
### Instruction:
Evaluate the statement by considering the following criteria derived from the ontology classes. Use your assessment to classify the statement as Hearsay or not.

Statement: {prompt_text}

Criteria:\n"""

    # Add filtered labels and definitions to the prompt
    filtered_words = remove_stopwords(prompt_text)
    filtered_labels = filter_words_by_substring(filtered_words,owl_df)
    filtered_labels.append(filter_label_by_substring(owl_df,'hearsay'))
    for label_definition in filtered_labels:
        full_prompt += f"\n{label_definition}"

    full_prompt += "\n\nOutput Format: One word (Yes/No) indicating whether the statement is hearsay"

    full_prompt += "\n\n### Response:\n"
    return full_prompt


# Testing - Hearsay

In [60]:
filtered_labels = filter_label_by_substring(owl_df, "hearsay")
filtered_labels

['Motion in Limine to Exclude Hearsay Witness : A Motion in Limine to Exclude Hearsay Witness is a legal request made by one party to prevent the other party from presenting testimony from a witness who will testify about statements made by someone else out of court, which are being offered to prove the truth of the matter asserted.',
 'Motion to Exclude Hearsay Witness : A Motion to Exclude Hearsay Witness is a legal request to prevent a witness from testifying in court based on the fact that their testimony is based on hearsay evidence.']

In [61]:
example_prompt = add_labels_and_definitions_to_prompt(prompts[1])
print(example_prompt)


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Evaluate the statement by considering the following criteria derived from the ontology classes. Use your assessment to classify the statement as Hearsay or not.

Statement: On the issue of whether Robert negligently drove, the fact that Robert fell asleep while driving.

Criteria:

Analysis of Pre-Petition Lien Issues : Analysis of Pre-Petition Lien Issues refers to the examination and evaluation of any legal claims or encumbrances on property that arise before the filing of a bankruptcy petition.
Travel Document Issued Date : The Travel Document Issued Date is the specific date on which a travel document like a passport or visa was officially issued by the relevant authority. This date is important for determining the document's validity period and is often required when applying for visas or entering foreign countries
JE10 Issue and Serve Proceedings and Prepar

In [66]:
prompt = add_labels_and_definitions_to_prompt(prompts[4])

response = get_llama_response(prompt,tokenizer,model)

print(response)

 No</s>


In [70]:
responses = []

filtered_labels = filter_label_by_substring(owl_df, "hearsay")

for i, prompt_text in enumerate(prompts):

    full_prompt = add_labels_and_definitions_to_prompt(prompt_text)

    response = get_llama_response(full_prompt,tokenizer,model)
    
    responses.append(response)
    
    print(f"Done for prompt {i+1}")
    # print(response)


Done for prompt 1
Done for prompt 2
Done for prompt 3
Done for prompt 4
Done for prompt 5
Done for prompt 6
Done for prompt 7
Done for prompt 8
Done for prompt 9
Done for prompt 10
Done for prompt 11
Done for prompt 12
Done for prompt 13
Done for prompt 14
Done for prompt 15
Done for prompt 16
Done for prompt 17
Done for prompt 18
Done for prompt 19
Done for prompt 20
Done for prompt 21
Done for prompt 22
Done for prompt 23
Done for prompt 24
Done for prompt 25
Done for prompt 26
Done for prompt 27
Done for prompt 28
Done for prompt 29
Done for prompt 30
Done for prompt 31
Done for prompt 32
Done for prompt 33
Done for prompt 34
Done for prompt 35
Done for prompt 36
Done for prompt 37
Done for prompt 38
Done for prompt 39
Done for prompt 40
Done for prompt 41
Done for prompt 42
Done for prompt 43
Done for prompt 44
Done for prompt 45
Done for prompt 46
Done for prompt 47
Done for prompt 48
Done for prompt 49
Done for prompt 50
Done for prompt 51
Done for prompt 52
Done for prompt 53
Do

error: missing ), unterminated subpattern at position 0

In [75]:
for i in range(len(responses)):
    print(responses[i])

 No</s>
 No</s>
 No</s>
 No</s>
 No</s>
 No</s>
 No</s>
 Hearsay</s>
 No</s>
 No</s>
 Hearsay</s>
 No</s>
 No</s>
 No</s>
 Hearsay</s>
 Yes</s>
 Hearsay</s>
 No</s>
 Hearsay</s>
 Yes</s>
 No</s>
 Yes</s>
 Hearsay</s>
 No</s>
 Yes</s>
 No</s>
 No</s>
 Hearsay</s>
 Yes</s>
 No</s>
 No</s>
 No</s>
 No</s>
 No</s>
 No</s>
 No</s>
 No</s>
 No</s>
 No</s>
 Hearsay</s>
 No</s>
 Hearsay</s>
 No</s>
 Hearsay</s>
 Hearsay: No, the statement is not hearsay. The statement is based on a document, an email to a mother explaining how her daughter was planning on purchasing a red car. The email is a primary source of information, and the statement is based on the information in the email.</s>
 No</s>
 Hearsay</s>
 No</s>
 Hearsay</s>
 Yes: ['Motion in Limine to Exclude Hearsay Witness : A Motion in Limine to Exclude Hearsay Witness is a legal request made by one party to prevent the other party from presenting testimony from a witness who will testify about statements made by someone else out of court

### Parsing output

In [76]:
import random

def extract_answers(original_responses):
    # Define replacements for specific expected endings
    replacements = {
        'Hearsay</s>': 'Yes',
        'No</s>': 'No'
    }
    
    # Create an empty list to store extracted answers
    extracted_answers = []

    # Process each response
    for response in original_responses:
        # Initially assume the response is not matched
        matched = False

        # Replace specific strings with predefined replacements if they match
        for key, value in replacements.items():
            if key in response:
                response = value
                matched = True
                break  # Exit the loop once a match is found

        # If no predefined response is matched, choose randomly between 'Yes' and 'No'
        if not matched:
            response = random.choice(['Yes', 'No'])

        # Append the modified response to the list
        extracted_answers.append(response)

    return extracted_answers


In [77]:
extracted_answers = extract_answers(responses)

In [78]:
print(extracted_answers)

['No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No']


In [79]:
evaluate("hearsay", extracted_answers, test_df["answer"].tolist()[:len(extracted_answers)])

0.5061188811188811